In [69]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
import random
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import gc
from scipy.stats import skew, boxcox
from scipy.stats.mstats import gmean
from scipy import sparse
from sklearn.metrics import log_loss
from datetime import datetime
import copy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 2017

In [70]:
data_path = "../input/"
train_X = pd.read_csv(data_path + 'train_BrandenMurray.csv')
test_X = pd.read_csv(data_path + 'test_BrandenMurray.csv')
train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))
ntrain = train_X.shape[0]
# all_features = features_to_use + desc_sparse_cols + feat_sparse_cols
print train_X.shape, test_X.shape, train_y.shape

(49352, 285) (74659, 285) (49352L,)


In [71]:
train_X.head()

,listing_id,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,...,manager_id_mean_med,manager_id_mean_high,desc_wordcount,pricePerBed,pricePerBath,pricePerRoom,bedPerBath,bedBathDiff,bedBathSum,bedsPerc
0,7170325,1.0,1,3387,281,40.7108,-73.9539,2104,2400.0,2947,...,0.403788,5.947230e-02,553,2400.0,2400.0,1200.000000,2400.0,0.0,2.0,0.500000
1,7092344,1.0,2,4758,3434,40.7513,-73.9722,1964,3800.0,8215,...,0.650389,5.004210e-02,827,1900.0,3800.0,1266.666667,3800.0,1.0,3.0,0.666667
2,7158677,1.0,2,5289,3457,40.7575,-73.9625,2627,3495.0,15314,...,0.194666,4.161561e-02,799,1747.5,3495.0,1165.000000,3495.0,1.0,3.0,0.666667
3,7211212,1.5,3,2104,4018,40.7145,-73.9425,1204,3000.0,21701,...,0.248812,6.214142e-31,588,1000.0,2000.0,666.666667,2000.0,1.5,4.5,0.666667
4,7225292,1.0,0,4930,3411,40.7439,-73.9743,610,2795.0,13511,...,0.140153,2.440795e-18,344,-1.0,2795.0,2795.000000,2795.0,-1.0,1.0,0.000000


In [72]:
def add_median_price(key=None, suffix="", trn_df=None, tst_df=None):
    # Set features to be used
    median_features = copy.copy(key)
    median_features.append('price')
    # Concat train and test to find median prices over whole dataset
    median_prices = pd.concat([trn_df[median_features], tst_df[median_features]], axis=0)
    # Group data by key to compute median prices
    medians_by_key = median_prices.groupby(by=key)['price'].median().reset_index()
    # Rename median column with provided suffix
    medians_by_key.rename(columns={'price': 'median_price_' + suffix}, inplace=True)
    # Update data frames, note that merge seems to reset the index
    # that's why I reset first and set again the index
    trn_df = trn_df.merge(medians_by_key, on=key, how='left')
    tst_df = tst_df.merge(medians_by_key, on=key, how='left')
    trn_df['ratio_' + suffix] = trn_df['price'] /trn_df['median_price_' + suffix]
    tst_df['ratio_' + suffix] = tst_df['price'] / tst_df['median_price_' + suffix]

    return trn_df, tst_df

In [73]:
train_X, test_X = add_median_price(key=['bedrooms'],
                               suffix="bed",
                               trn_df=train_X, tst_df=test_X)

In [74]:
train_X.head()

,listing_id,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,...,desc_wordcount,pricePerBed,pricePerBath,pricePerRoom,bedPerBath,bedBathDiff,bedBathSum,bedsPerc,median_price_bed,ratio_bed
0,7170325,1.0,1,3387,281,40.7108,-73.9539,2104,2400.0,2947,...,553,2400.0,2400.0,1200.000000,2400.0,0.0,2.0,0.500000,2900.0,0.827586
1,7092344,1.0,2,4758,3434,40.7513,-73.9722,1964,3800.0,8215,...,827,1900.0,3800.0,1266.666667,3800.0,1.0,3.0,0.666667,3350.0,1.134328
2,7158677,1.0,2,5289,3457,40.7575,-73.9625,2627,3495.0,15314,...,799,1747.5,3495.0,1165.000000,3495.0,1.0,3.0,0.666667,3350.0,1.043284
3,7211212,1.5,3,2104,4018,40.7145,-73.9425,1204,3000.0,21701,...,588,1000.0,2000.0,666.666667,2000.0,1.5,4.5,0.666667,4500.0,0.666667
4,7225292,1.0,0,4930,3411,40.7439,-73.9743,610,2795.0,13511,...,344,-1.0,2795.0,2795.000000,2795.0,-1.0,1.0,0.000000,2400.0,1.164583


In [75]:
test_X.head()

,listing_id,bathrooms,bedrooms,building_id,display_address,latitude,longitude,manager_id,price,street_address,...,desc_wordcount,pricePerBed,pricePerBath,pricePerRoom,bedPerBath,bedBathDiff,bedBathSum,bedsPerc,median_price_bed,ratio_bed
0,7142618,1.0,1,3081,4452,40.7185,-73.9865,2335,2950,23000,...,587,2950.0,2950.0,1475.000000,2950.0,0.0,2.0,0.500000,2900.0,1.017241
1,7210040,1.0,2,2,4502,40.7278,-74.0000,2727,2850,4942,...,245,1425.0,2850.0,950.000000,2850.0,1.0,3.0,0.666667,3350.0,0.850746
2,7174566,1.0,0,2,4458,40.7260,-74.0026,3005,2295,1267,...,268,-1.0,2295.0,2295.000000,2295.0,-1.0,1.0,0.000000,2400.0,0.956250
3,7191391,1.0,2,6401,3861,40.7321,-74.0028,870,2900,8106,...,146,1450.0,2900.0,966.666667,2900.0,1.0,3.0,0.666667,3350.0,0.865672
4,7171695,1.0,1,3267,3624,40.7054,-74.0095,2187,3254,5730,...,564,3254.0,3254.0,1627.000000,3254.0,0.0,2.0,0.500000,2900.0,1.122069


In [78]:
train_X.to_csv(data_path + 'train_BrandenMurray_MedianBedroom.csv', index = False)
test_X.to_csv(data_path + 'test_BrandenMurray_MedianBedroom.csv', index = False)